# Goodreads

En este proyecto utilizaré el API de la página Goodreads para recopilar la información de mi perfil. En total, me interesa generar dos bases de datos: una sobre los libros que leído y otra sobre los libros que quiero leer. 

NOTA: Con respecto a los términos y condiciones del API de la página, debo ser cuidadoso con uno en particular: "Not request any method more than once a second. Goodreads tracks all requests made by developers".

#### Algunos aspectos que me gustaría incluir después:
- La información de otros usuarios cuya opinión respeto.
- Extraer información de todos los usuarios de México.
- Un programa que me sugiera cuál libro debo leer, considerando mis preferencias y las recomendaciones de los usuarios a los que sigo. 

#### Mejoras pendientes a mi código:
- Manejar errores, con try/except, en los GET requests.
- Optimizar mi función de tags, para que no haga un GET por cada tag, sino un solo GET que se puede usar para todas las tags. Si no, el proceso es lentísimo y creo que le puedo aumentar la velocidad considerablemente.
- Agregar función para determinar pages (múltiplos de 200). Así, solo hay que decirle el shelf size. Sin embargo, habría que ver si en la API hay forma de ver el shelf size automáticamente.

## Obtener información de mi perfil

Comienzo por importar las librerías que utilizaré:

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

A su vez, defino algunas variables importantes que utilizaré en todo el proceso, incluyendo el key que generé directamente en la página de Goodreads:

In [2]:
CONSUMER_KEY = 'PnnwwD2CW9RtBv6YqsjYw'
CONSUMER_SECRET = 'S1jfjAZpNn1dH6iRBWgDINoFp8G2ucwJa1C2OHg70xw'
url = "https://www.goodreads.com/"
my_id = '40732498'

### 1. Obtener código html de libreros

Con la siguiente función hago un get request para obtener el código html de un librero. La función permite seleccionar el librero que me interesa, ya sea el de libros leídos (`'read'`) o de libros que quiero leer (`'to-read'`). 

Es importante recalcar que la función solo obtiene el códido de UNA página que despliega 200 libros. Es decir, si en el librero hay más de 200, hay que volver a llamar la función pero especificándole que queremos la segunda página o alguna sucesiva.

In [3]:
def shelf_info(user_id, shelf, page):
  
    """Arroja un BeautifulSoup object de una página del librero de un usuario"""
    
    time.sleep(1.1) # Para cumplir con los términos y condiciones de Goodreads
    info = requests.get(f'{url}/review/list?v=2&id={user_id}&shelf={shelf}&sort=title&page={page}&per_page=200&key={CONSUMER_KEY}')
    print('Status code: ', info.status_code)
    info_content = info.content
    soup = BeautifulSoup(info_content, 'lxml')
    return soup

### 2. Explorar los tags relevantes

Exploremos el código html de un libro para saber qué tags deberemos utilizar en nuestra extracción posterior de datos. 

In [4]:
leidos_p1 = shelf_info(my_id, 'read', 1)

Status code:  200


In [5]:
print(leidos_p1.prettify())

<?xml version="1.0" encoding="UTF-8"?>
<html>
 <body>
  <goodreadsresponse>
   <request>
    <authentication>
     true
    </authentication>
    <key>
    </key>
    <method>
    </method>
   </request>
   <shelf exclusive="true" id="132637058" name="read" sortable="false">
   </shelf>
   <reviews end="200" start="1" total="276">
    <review>
     <id>
      1413022382
     </id>
     <book>
      <id type="integer">
       47668
      </id>
      <isbn>
       9707311150
      </isbn>
      <isbn13>
       9789707311152
      </isbn13>
      <text_reviews_count type="integer">
       1053
      </text_reviews_count>
      <uri>
       kca://book/amzn1.gr.book.v1.bmTGkHHUHoKuL_OJ3dmk7w
      </uri>
      <title>
       Ensayo Sobre la Ceguera
      </title>
      <title_without_series>
       Ensayo Sobre la Ceguera
      </title_without_series>
      <image_url>
       https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1307808756l/47668._SX98_.jpg
      </image_url

Podemos ver que el registro de cada libro comienza con la etiqueta `review`. Imprimamos el código del primer libro de la lista para identificar mejor las etiquetas:

In [6]:
print(leidos_p1.find('review').prettify())

<review>
 <id>
  1413022382
 </id>
 <book>
  <id type="integer">
   47668
  </id>
  <isbn>
   9707311150
  </isbn>
  <isbn13>
   9789707311152
  </isbn13>
  <text_reviews_count type="integer">
   1053
  </text_reviews_count>
  <uri>
   kca://book/amzn1.gr.book.v1.bmTGkHHUHoKuL_OJ3dmk7w
  </uri>
  <title>
   Ensayo Sobre la Ceguera
  </title>
  <title_without_series>
   Ensayo Sobre la Ceguera
  </title_without_series>
  <image_url>
   https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1307808756l/47668._SX98_.jpg
  </image_url>
  <small_image_url>
   https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1307808756l/47668._SY75_.jpg
  </small_image_url>
  <large_image_url>
  </large_image_url>
  <link/>
  https://www.goodreads.com/book/show/47668.Ensayo_Sobre_la_Ceguera
  <num_pages>
   329
  </num_pages>
  <format>
   Paperback
  </format>
  <edition_information>
   1st edition
  </edition_information>
  <publisher>
   Punto de Lectura
  </publisher>


No toda la información del libro me interesa. A continuación incluyo las tags que sí me interesan:

In [7]:
all_tags = ['id', 'isbn', 'title', 'link/', 'num_pages', 'publisher', 'publication_year', 'average_rating', 'ratings_count', 'name', 'rating']

En lo que sigue, debo ser cuidadoso con las tags que se repiten, como `average_rating` y `ratings_count`, puesto que es la información tanto del libro como del autor del libro en específico. Quizá por eso mejor conviene poner esas etiquetas por separado.

In [26]:
tags_unique = ['isbn', 'title', 'num_pages', 'publisher', 'publication_year', 'name', 'rating']
tags_repetidas = ['average_rating', 'ratings_count']

### 3. Extraer información del librero de libros leídos

La siguiente función permite extraer la información de una etiqueta de un librero.  Con ella, después podremos iterar sobre las tags que nos interesan. Sin embargo, la función solo es útil para tags que no están repetidas.

In [9]:
def get_tag_info(user_id, tag, shelf, pages=2):
    """Arroja una lista con la información específica de una etiqueta del librero de un usuario"""
    
    toda_info = []
    
    for x in range(1, pages+1):
        page = shelf_info(user_id, shelf=shelf, page=x)
        page_info = page.find_all(f'{tag}')
        page_info = [elem.get_text() for elem in page_info]
        toda_info.append(page_info)
    
    toda_info_clean = [elem for list in toda_info for elem in list]
    return toda_info_clean

Procedamos ahora a definir una función para extraer la información de cada etiqueta y organizarla dentro de un diccionario.

In [10]:
def iterador_tags(user_id, tags, shelf, pages=2):
    """Devuelve un diccionario con la información de tags"""
    
    diccionario = {}
       
    for tag in tags:
        informacion_tag = get_tag_info(user_id, tag, shelf, pages=pages)
        diccionario[f'{tag}'] = informacion_tag
        
    return diccionario

In [11]:
información_tags = iterador_tags(my_id, tags_unique, 'read')

Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200


Aún falta, no obstante, extraer la información de las etiquetas que estaban repetidas. Para ello, debemos alterar ligeramente la función de `get_info()`. Al ver el código html, observo que solo me interesa la primera vez en que aparecen estas etiquetas repetidas (es decir, la información referente al libro, no al autor del libro). 

#### a) Obtener `average_rating` y `ratings_count`

Utilizaré la siguiente función para estas etiquetas que aparecen repetidas:

In [13]:
def get_info_repeated_tags(user_id, tag, shelf, pages=2):
    
    """Arroja información específica del librero de un usuario, para etiquetas repetidas"""
    
    toda_info = []
    
    for x in range(1, pages+1):
        page = shelf_info(user_id, shelf=shelf, page=x)
        review_blocks = page.find_all('review')
        
        for review in review_blocks: 
            info = review.find(f'{tag}').get_text()
            toda_info.append(info)

    return toda_info

Utilicemos la función con dos de nuestras variables repetidas:

In [25]:
tags_repetidas[0]

'link/'

In [27]:
average_ratings = get_info_repeated_tags(my_id, tags_repetidas[0], 'read')
ratings_counts = get_info_repeated_tags(my_id, tags_repetidas[1], 'read')

Status code:  200
Status code:  200
Status code:  200
Status code:  200


Agreguémoslas a nuestro diccionario anterior:

In [15]:
if 'average_ratings' not in información_tags:
    información_tags['average_ratings'] = average_ratings

if 'ratings_count' not in información_tags:
    información_tags['ratings_count'] = ratings_counts


#### b) Obtener links

Debemos utilizar una función diferente para extraer los links, puesto que no identifico que tengan una etiqueta como tal en el código html. Para ello, utilizaré regex:

In [16]:
def get_links(user_id, shelf, pages=2):
    
    """Arroja los links de librero de un usuario"""
    
    toda_info = []
    link_pattern = re.compile(r'www.goodreads.com.*')
    
    for x in range(1, pages+1):
        page = shelf_info(user_id, shelf=shelf, page=x)
        review_blocks = page.find_all('review')
        
        for review in review_blocks: 
            if link_pattern.search(review.get_text()):
                link = re.findall(link_pattern, review.get_text()) 
                toda_info.append(link)
    
            else: 
                print('Missing: ', review.title)
    
    toda_info_flatten = [list[0] for list in toda_info]
                
    return toda_info_flatten

In [17]:
links = get_links(my_id, 'read', pages=2)

Status code:  200
Status code:  200


Ahora agregamos la información al diccionario principal:

In [18]:
if 'link' not in información_tags:
    información_tags['link'] = links

## Juntar todo en una función

Para que sea más sencillo extraer la información de cualquier usuario, juntemos todas las funciones que utilizamos en la seción anterior para que nos quede en una sola función:

In [49]:
def get_user_shelf_info(user_id, shelf, pages):
    """Regresa un diccionario con la información de todas las etiquetas de un librero"""
    
    tags_unique = ['isbn', 'title', 'num_pages', 'publisher', 'publication_year', 'name', 'rating']
    tags_repetidas = ['average_rating', 'ratings_count']
    
    informacion_tags = iterador_tags(user_id, tags_unique, shelf, pages=pages)
    average_ratings = get_info_repeated_tags(user_id, tags_repetidas[0], shelf, pages=pages)
    ratings_counts = get_info_repeated_tags(user_id, tags_repetidas[1], shelf, pages=pages)
    links = get_links(user_id, shelf, pages=pages)
    
    if 'average_ratings' not in informacion_tags:
        informacion_tags['average_ratings'] = average_ratings

    if 'ratings_count' not in informacion_tags:
        informacion_tags['ratings_count'] = ratings_counts
    
    if 'link' not in informacion_tags:
        informacion_tags['link'] = links
    
    if 'user_id' not in informacion_tags:
        informacion_tags['user_id'] = [user_id for x in range(0, len(average_ratings))]
    
    if 'librero' not in informacion_tags:
        informacion_tags['librero'] = [shelf for x in range(0, len(average_ratings))]
    
    return informacion_tags

### Información del usuario 'Francisco Galán'

#### a) Estante de libros leídos

In [50]:
id_francisco_galan = '40732498'
leido_francisco_galan = get_user_shelf_info(id_francisco_galan, 'read', pages=2)

Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200


#### b) Estante de libros por leer

In [51]:
por_leer_francisco_galan = get_user_shelf_info(id_francisco_galan, 'to_read', pages=2)

Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200
Status code:  200


## Generar databases en pandas

Antes de pasar la información a formato DataFrame en pandas, chequemos que todas las variables tengan la misma extensión:

In [52]:
def check_variables_length(dictionary):
    for x, y in dictionary.items():
        print(f'{x}: ', len(y))

In [53]:
check_variables_length(leido_francisco_galan)

isbn:  276
title:  276
num_pages:  276
publisher:  276
publication_year:  276
name:  276
rating:  276
average_ratings:  276
ratings_count:  276
link:  276
user_id:  276
librero:  276


Ahora sí, pasemos los datos a un DataFrame

In [54]:
pd.DataFrame(leido_francisco_galan)

,isbn,title,num_pages,publisher,publication_year,name,rating,average_ratings,ratings_count,link,user_id,librero
0,9707311150,Ensayo Sobre la Ceguera,329,Punto de Lectura,2006,José Saramago,4,4.14,202539,www.goodreads.com/book/show/47668.Ensayo_Sobre...,40732498,read
1,,1984,328,New American Library,1950,George Orwell,4,4.19,3165222,www.goodreads.com/book/show/5470.1984,40732498,read
2,0451457994,"2001: A Space Odyssey (Space Odyssey, #1)",297,Roc,2000,Arthur C. Clarke,3,4.15,261264,www.goodreads.com/book/show/70535.2001,40732498,read
3,0307465357,"The 4-Hour Workweek: Escape 9-5, Live Anywhere...",396,Harmony,2009,Timothy Ferriss,3,3.89,184788,www.goodreads.com/book/show/6444424-the-4-hour...,40732498,read
4,9380227884,8 to be Great: The 8 Traits Successful People ...,,,,Richard St. John,2,4.07,415,www.goodreads.com/book/show/19761002-8-to-be-g...,40732498,read
...,...,...,...,...,...,...,...,...,...,...,...,...
271,8478718575,Y no quedó ninguno,222,Rba Libros,2007,Agatha Christie,4,4.26,831677,www.goodreads.com/book/show/2468267.Y_no_qued_...,40732498,read
272,,Your Move: The Underdog's Guide to Building Yo...,,,,Ramit Sethi,3,4.00,775,www.goodreads.com/book/show/35202699-your-move,40732498,read
273,,Your Perfect Right: Assertiveness and Equality...,,,,Robert Alberti,1,3.75,404,www.goodreads.com/book/show/664676.Your_Perfec...,40732498,read
274,8490625662,¿Quién mató a Palomino Molero?,176,Debolsillo,2015,Mario Vargas Llosa,3,3.55,4998,www.goodreads.com/book/show/25785265-qui-n-mat...,40732498,read


In [55]:
check_variables_length(por_leer_francisco_galan)

isbn:  382
title:  382
num_pages:  382
publisher:  382
publication_year:  382
name:  382
rating:  382
average_ratings:  382
ratings_count:  382
link:  382
user_id:  382
librero:  382


In [56]:
pd.DataFrame(por_leer_francisco_galan)

,isbn,title,num_pages,publisher,publication_year,name,rating,average_ratings,ratings_count,link,user_id,librero
0,9707311150,Ensayo Sobre la Ceguera,329,Punto de Lectura,2006,José Saramago,4,4.14,202539,www.goodreads.com/book/show/47668.Ensayo_Sobre...,40732498,to_read
1,,1984,328,New American Library,1950,George Orwell,4,4.19,3165222,www.goodreads.com/book/show/5470.1984,40732498,to_read
2,0451457994,"2001: A Space Odyssey (Space Odyssey, #1)",297,Roc,2000,Arthur C. Clarke,3,4.15,261264,www.goodreads.com/book/show/70535.2001,40732498,to_read
3,0307465357,"The 4-Hour Workweek: Escape 9-5, Live Anywhere...",396,Harmony,2009,Timothy Ferriss,3,3.89,184788,www.goodreads.com/book/show/6444424-the-4-hour...,40732498,to_read
4,0578522829,The 5 Mistakes Every Investor Makes and How to...,194,Peter Mallouk,2014,Peter Mallouk,0,4.31,348,www.goodreads.com/book/show/46255785-the-5-mis...,40732498,to_read
...,...,...,...,...,...,...,...,...,...,...,...,...
377,,Your Money or Your Life,366,Penguin Books,,Vicki Robin,0,4.07,17882,www.goodreads.com/book/show/43560266-your-mone...,40732498,to_read
378,,Your Move: The Underdog's Guide to Building Yo...,,,,Ramit Sethi,3,4.00,775,www.goodreads.com/book/show/35202699-your-move,40732498,to_read
379,,Your Perfect Right: Assertiveness and Equality...,,,,Robert Alberti,1,3.75,404,www.goodreads.com/book/show/664676.Your_Perfec...,40732498,to_read
380,8490625662,¿Quién mató a Palomino Molero?,176,Debolsillo,2015,Mario Vargas Llosa,3,3.55,4998,www.goodreads.com/book/show/25785265-qui-n-mat...,40732498,to_read
